In [19]:
import math
from catboost import Pool, CatBoostClassifier, CatBoostRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [10]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats (containers with only __len__ and __getitem__ defined).
        # weights parameter can be None.
        # Returns list of pairs (der1, der2)
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        exponents = []
        for index in range(len(approxes)):
            exponents.append(math.exp(approxes[index]))

        result = []
        for index in range(len(targets)):
            p = exponents[index] / (1 + exponents[index])
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))

        return result

In [11]:
X, y = make_classification(n_samples=50, n_features=4, n_informative=2)

In [12]:
xtr, xt, ytr, yt = train_test_split(X, y)

In [13]:
train_pool = Pool(xtr, ytr)
test_pool = Pool(xt)

In [14]:
model = CatBoostClassifier(iterations=1, loss_function=LoglossObjective(), eval_metric="Logloss")

In [16]:
model.fit(train_pool, logging_level='Verbose')

0:	learn: 0.6883170	total: 6.35ms	remaining: 0us


In [17]:
preds_raw = model.predict(test_pool, prediction_type='RawFormulaVal') 

In [18]:
preds_raw

array([-0.00461538,  0.00461538, -0.024     ,  0.015     ,  0.00352941,
        0.004     ,  0.00461538, -0.024     ,  0.015     , -0.024     ,
        0.015     , -0.024     ,  0.        ])

In [ ]:
CatBoostRegressor(loss_function=)